# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [37]:
import pandas as pd
import pymongo

In [38]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [39]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [23]:
df_youtube.dtypes

video_id           object
title              object
channel_title      object
category_id         int64
tags               object
views               int64
likes               int64
dislikes            int64
comment_total       int64
thumbnail_link     object
date              float64
dtype: object

In [18]:
# Question 3 (tags to list)
# df_youtube['tags'] = df_youtube['tags'].str.split("|")

In [24]:
df_youtube['tags']

0       logan paul vlog|logan paul|logan|paul|olympics...
1       Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...
2                                                  [none]
3       apple iphone x hands on|Apple iPhone X|iPhone ...
4       jacksfilms|parody|parodies|iphone|iphone x|iph...
                              ...                        
7987    sinkhole|how do sinkholes form|sinkhole in wat...
7988    Business Insider|Donald Trump|Hillary Clinton|...
7989    LP|Death Valley|Other People|Lost On You|The M...
7990    YouTube FanFest|#YTFF|Washington DC|USA|YTFF|#...
7991    LIGO|Virgo|collaboration|space|science|nationa...
Name: tags, Length: 7992, dtype: object

In [40]:
data_dict = df_youtube.to_dict("records")

### Importer les données

In [148]:
#Si besoin : 
collection.delete_many({})

In [149]:
collection.insert_many(data_dict)

## Question 1  
Récupérer toutes les vidéos de la chaîne Apple.

In [150]:
apple_vids = collection.find({"channel_title":"Apple"})
list(apple_vids)

[{'_id': ObjectId('61ddc489e7ed7776db2c988b'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09},
 {'_id': ObjectId('61ddc489e7ed7776db2c9955'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 12200526,
  'likes': 258842,
  'dislikes': 44339,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 14.09},
 {'_id': Obje

## Question 2
Compter le nombre de catégories différentes

In [151]:
categories = collection.distinct("category_id")
len(categories)

16

## Question 3
Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

In [152]:
collection.find_one()

{'_id': ObjectId('61ddc489e7ed7776db2c988a'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09}

In [153]:
collection.update_many({}, [{"$set": {"tags": { "$split": ["$tags", "|"]}}}])
collection.find_one()

{'_id': ObjectId('61ddc489e7ed7776db2c988a'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09}

## Question 4
Récupérer les vidéos les plus vues.

In [158]:
number = 10 # Combien de video afficher
most_viewed = collection.find().sort([("views", -1)]).limit(number)
list(most_viewed)

[{'_id': ObjectId('61ddc489e7ed7776db2c9ffd'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': ObjectId('61ddc489e7ed7776db2c9f28'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09},
 {'_id': ObjectId('61ddc489e7ed7776db2c993b'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags'

## Question 5
Compter le nombre moyen de vues en fonction de la catégorie.

In [167]:
views_cat = collection.aggregate([{"$group":{ "_id": "$category_id", "mean": { "$avg": "$views" } } },{"$sort": {"mean":-1}}])
list(views_cat)

[{'_id': 23, 'mean': 1240073.294039735},
 {'_id': 10, 'mean': 1176553.612},
 {'_id': 24, 'mean': 1154868.4397251718},
 {'_id': 29, 'mean': 1110334.2142857143},
 {'_id': 1, 'mean': 1039472.6560846561},
 {'_id': 22, 'mean': 971532.7573696146},
 {'_id': 28, 'mean': 924730.548828125},
 {'_id': 17, 'mean': 728434.3243902439},
 {'_id': 20, 'mean': 681081.8170731707},
 {'_id': 15, 'mean': 651404.3879310344},
 {'_id': 2, 'mean': 607693.8706896552},
 {'_id': 27, 'mean': 547582.4700598803},
 {'_id': 25, 'mean': 540955.5569823436},
 {'_id': 26, 'mean': 537665.1806674339},
 {'_id': 19, 'mean': 464041.0833333333},
 {'_id': 43, 'mean': 8492.5}]

## Question 6 
Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [171]:
chan_likes = collection.aggregate([{"$group":{ "_id": "$channel_title", "mean": { "$avg": "$likes" } } }, {"$sort": {"mean":-1}}])
list(chan_likes)

[{'_id': 'ZaynVEVO', 'mean': 1431683.0},
 {'_id': 'ibighit', 'mean': 1371766.25},
 {'_id': 'melanie martinez', 'mean': 911871.5},
 {'_id': 'BETNetworks', 'mean': 769687.625},
 {'_id': 'jypentertainment', 'mean': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'mean': 653443.5384615385},
 {'_id': 'Wengie', 'mean': 567235.2},
 {'_id': 'Alesso', 'mean': 509417.28571428574},
 {'_id': 'PewDiePie', 'mean': 508647.8},
 {'_id': 'shakiraVEVO', 'mean': 434302.14285714284},
 {'_id': 'Ed Sheeran', 'mean': 395654.1666666667},
 {'_id': 'Dude Perfect', 'mean': 389697.1111111111},
 {'_id': 'Conor Maynard', 'mean': 389273.0},
 {'_id': 'TaylorSwiftVEVO', 'mean': 367455.6666666667},
 {'_id': 'Liza Koshy', 'mean': 365146.9583333333},
 {'_id': 'Linkin Park', 'mean': 356475.7272727273},
 {'_id': 'jbalvinVEVO', 'mean': 345130.4285714286},
 {'_id': 'Primitive Technology', 'mean': 340659.25},
 {'_id': 'Gabbie Hanna', 'mean': 339488.0},
 {'_id': 'Bhad Bhabie', 'mean': 314148.3333333333},
 {'_id': 'ImagineDrago